# Sample 3

This notebook demonstrate how to use Azure Eval SDK to run evaluations in the cloud (AI Foundry).

In [ ]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.evaluation import RelevanceEvaluator
from custom_evaluator.friendliness_evaluator import FriendlinessEvaluator

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from promptflow.client import PFClient

from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType
)
from azure.mgmt.machinelearningservices import AzureMachineLearningWorkspaces


In [ ]:
# optional - to support DefaultAzureCredential()
!az login

In [ ]:
load_dotenv()

### Upload dataset to AI Foundry

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AI_FOUNDRY_CONNECTION_STRING"]
)

In [ ]:
file_path = os.path.abspath("data/qna.jsonl")

try:
    if os.path.exists(file_path):
        print(f"File found: {file_path}")
        print(f"File size: {os.path.getsize(file_path)} bytes")
        data_id, _ = project_client.upload_file(file_path)
        print(f"Data ID: {data_id}")
except Exception as e:
    print(f"Error uploading file: {e}")

### Register the custom evaluator into AI Foundry
Steps 
- Convert the evaluator into an evaluation flow and save locally 
- Register it to AI Foundry

In [ ]:
ml_client = MLClient(
       subscription_id=os.environ["SUBSCRIPTION_ID"],
       resource_group_name=os.environ["RESOURCE_GROUP"],
       workspace_name=os.environ["AI_FOUNDRY_PROJECT_NAME"],
       credential=DefaultAzureCredential()
)

# Create flow from the custom evaluator
local_path = "dist/FriendlinessEvaluator" # save this evaluation flow in /dist
pf_client = PFClient()
pf_client.flows.save(entry=FriendlinessEvaluator, path=local_path)

evaluator_name = "FriendlinessEvaluator"
custom_evaluator = Model(
    path=local_path,
    name=evaluator_name,
    description="custom prompt evaluator measuring response base on friendliness",
)

# Register the custom evaluator
registered_evaluator = ml_client.evaluators.create_or_update(custom_evaluator)
print("Registered evaluator id:", registered_evaluator.id)

versioned_evaluator = ml_client.evaluators.get(evaluator_name, label="latest")
print("Versioned evaluator id:", versioned_evaluator.version)

### Programatically construct custom evaluator id

Currently when we register the custom evaluator into AI Foundry, the `registered_evaluator.id` variable returns the resource id (https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.job?view=azure-python) which is not in the format of the required asset ID in the parameter for `EvaluatorConfiguration` class. 
So there is two options to retrieve this asset ID:
- We can manually go into the AI Foundry Evaluator Library and copy the asset ID via UI.
- The format for the asset ID currently is `azureml://locations/{location}/workspaces/{project_id}/models/{evaluator_name}/versions/{versioned_evaluator.version}` so we can programmatically retrieve the project_id, evaluator_name and version of latest evaluator and construct the asset id for the custom evaluator. We will need to use `azure.mgmt.machinelearningservices` to retrieve the location and project_id that constructs the asset ID

In [ ]:
default_connection = project_client.connections.get_default(connection_type=ConnectionType.AZURE_OPEN_AI)

# Create model configuration for evaluators that need a model
model_config = default_connection.to_evaluator_model_config(
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

credential = DefaultAzureCredential()
subscription_id = os.environ["SUBSCRIPTION_ID"]

ml_client = AzureMachineLearningWorkspaces(credential, subscription_id)

resource_group_name = os.environ["RESOURCE_GROUP"]
workspace_name = os.environ["AI_FOUNDRY_PROJECT_NAME"]

workspace = ml_client.workspaces.get(resource_group_name, workspace_name)

project_id = workspace.workspace_id
location = workspace.location

custom_evaluator_id = f"azureml://locations/{location}/workspaces/{project_id}/models/{evaluator_name}/versions/{versioned_evaluator.version}"
print(custom_evaluator_id)

### Create evaluation using published custom evaluator library

In [ ]:
# Create an evaluation
evaluation = Evaluation(
    display_name="Cloud Evaluation 2",
    description="Cloud Evaluation",
    data=Dataset(id=data_id),
    evaluators={
        "relevance": EvaluatorConfiguration(
            id=RelevanceEvaluator.id, # in-built evaluator id
            init_params={
                "model_config": model_config
            },
            data_mapping={
                "query": "${data.question}",
                "response": "${data.answer}",
            }
        ),
        "friendliness": EvaluatorConfiguration(
            id=custom_evaluator_id, # custom evaluator id
            init_params={
                "model_config": model_config
            },
            data_mapping={
                "response": "${data.answer}",
            }
        ),

    },
)

evaluation_response = project_client.evaluations.create(
    evaluation=evaluation
)